## *Importing Modules*

In [93]:
! pip install bs4
! pip install selenium

grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## *Rendering Dynamic Consumer Product Website*

In [214]:
from selenium import webdriver
import bs4 as bs

PATH = "C:\Users\HP\Documents\chromedriver_win32\chromedriver.exe"
URL = "https://www.daraz.pk/catalog/?_keyori=ss&from=input&page=1&q=toys&spm=a2a0e.searchlistcategory.search.go.2d162a96soiFLM"

driver = webdriver.Chrome(PATH)
driver.get(URL)

## *Scraping Product Features*

In [215]:
titles , prices , original_prices , sale , discount = [],[],[],[],[]

def get_title(product):
    return product.find_all('div',class_="c16H9d")[0].find_all('a')[0].get("title")

def get_price(product):
    return product.find_all('span',class_="c13VH6")[0].text

def get_original_price(product):
    try:
        return product.find_all('del',class_="c13VH6")[0].text
    except:
        return get_price(product)

def get_discount_percent(product):
    try:
        return product.find_all('span',class_="c1hkC1")[0].text
    except:
        return '0'

def is_sale(product):
    try:
        return '1' if len(product.find_all('span',class_="c1hkC1")[0].text) >= 1 else '0'
    except:
        return '0'
    

## *Automation with Selenium to span across all pages*

In [216]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time

titles , prices , original_prices , sale , discount = [],[],[],[],[]
DELAY , MAX_PAGE_LIMIT = 20,50
count = 1 


for page_num in range(MAX_PAGE_LIMIT):
            try:
                soup = bs.BeautifulSoup(driver.page_source,'lxml')
                product_containers = soup.find_all('div',class_="c3e8SH")
                
                for product in product_containers:
                        titles.append(get_title(product))
                        prices.append(get_price(product))
                        original_prices.append(get_original_price(product))
                        discount.append(get_discount_percent(product))
                        sale.append(is_sale(product))
                
                print "Page ", count, "scrapped...."
                next_link = WebDriverWait(driver, DELAY).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.ant-pagination-next')))
                next_link.click()
                time.sleep(0.5)
                driver.get(driver.current_url)
                count = count + 1
                
            except NoSuchElementException:
                print "All pages successfully scrapped...."

print("All pages scrapped successfully...")

Page  1 scrapped....
Page  2 scrapped....
Page  3 scrapped....
Page  4 scrapped....
Page  5 scrapped....
Page  6 scrapped....
Page  7 scrapped....
Page  8 scrapped....
Page  9 scrapped....
Page  10 scrapped....
Page  11 scrapped....
Page  12 scrapped....
Page  13 scrapped....
Page  14 scrapped....
Page  15 scrapped....
Page  16 scrapped....
Page  17 scrapped....
Page  18 scrapped....
Page  19 scrapped....
Page  20 scrapped....
Page  21 scrapped....
Page  22 scrapped....
Page  23 scrapped....
Page  24 scrapped....
Page  25 scrapped....
Page  26 scrapped....
Page  27 scrapped....
Page  28 scrapped....
Page  29 scrapped....
Page  30 scrapped....
Page  31 scrapped....
Page  32 scrapped....
Page  33 scrapped....
Page  34 scrapped....
Page  35 scrapped....
Page  36 scrapped....
Page  37 scrapped....
Page  38 scrapped....
Page  39 scrapped....
Page  40 scrapped....
Page  41 scrapped....
Page  42 scrapped....
Page  43 scrapped....
Page  44 scrapped....
Page  45 scrapped....
Page  46 scrapped..

## *Data Transformation*

In [217]:
def data_transformation_numerical(data):
    return [int(re.findall(r'\d+',val.encode("utf-8"))[0])  for val in data]

def data_transformation_textual(data):
    return [val.encode("utf-8") for val in data]
    

prices_transformed = data_transformation_numerical(prices)
original_prices_transformed = data_transformation_numerical(original_prices)
discount_transformed = data_transformation_numerical(discount)
titles_transformed = data_transformation_textual(titles)



## *Tabulating Processed Data*

In [218]:
import pandas as pd

data_scrapped = pd.DataFrame(list(zip(titles_transformed, prices_transformed,sale,original_prices_transformed,discount_transformed)), 
               columns =['Title', 'Price','Sale','Original Price','Discount'])
data_scrapped.head()

,Title,Price,Sale,Original Price,Discount
0,Three In One (Jigsaw+Coloring Pages+Water Colo...,240,0,240,0
1,"Tool set, 17 pieces (case)",1,1,2,32
2,Pack Of 10 Toys Of Kitchen set For Kids Best Q...,149,1,400,63
3,Kids Remote Control Car,199,1,250,20
4,Pack Of -14 Pcs Mini Coffee Set Toy For Kids w...,99,1,150,34


## *Exporting Features*

In [219]:
import os

FILE_NAME = 'data_scrapped.csv'
DIRECTORY = "C:\Users\HP\Documents\chromedriver_win32"

if not os.path.exists(DIRECTORY):
    os.mkdir(DIRECTORY)

OUTPUT_FILE_PATH = os.path.join(DIRECTORY, FILE_NAME)    
data_scrapped.to_csv(OUTPUT_FILE_PATH,index=False,header=True)